In [1]:
import os
str1 = ''
os.chdir(r"E:\7.3徐州网赌\Q9案8.4反馈数据（64个账户）")
file_list = []
for path_name, dir, files_name in os.walk(os.getcwd()):
    for file in files_name:
        if str1 in file:
            file_list.append(os.path.join(path_name, file))
print(len(file_list))
file_list

12


['E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\上海浦东发展银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\中信银行交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\中国光大银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\中国建设银行交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\中国民生银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\中国银行交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\交通银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\人员联系方式信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\兴业银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\华夏银行股份有限公司交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\平安银行交易明细信息.csv',
 'E:\\7.3徐州网赌\\Q9案8.4反馈数据（64个账户）\\招商银行股份有限公司交易明细信息.csv']

In [2]:
import pandas as pd
df_group = pd.DataFrame(columns = ['Path','Columns','False','True'])
type1 = '.xls'
type2 = '.csv'
for i in range(len(file_list)):
#     print(i)
    if type1 in file_list[i]:
        df = pd.read_excel(file_list[i], encoding = "gbk")
    elif type2 in file_list[i]:
        df = pd.read_csv(file_list[i], engine = "python")
    col_name = df.columns
    bool_S = df.duplicated().value_counts()
    if len(bool_S)==1:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':0}
    elif len(bool_S)==2:
        df_group.loc[i] = {'Path':file_list[i],'Columns':tuple(col_name),'False':bool_S[0],'True':bool_S[1]}
    
grouped = df_group.groupby('Columns')
groupname_list = list(grouped.size().index)
print('以下'+str(len(groupname_list))+'种格式数据分类，保留流水、账户类别')
print(grouped.size())

以下2种格式数据分类，保留流水、账户类别
Columns
(交易卡号\t, 交易账号\t, 查询反馈结果原因\t, 交易户名\t, 交易证件号码\t, 交易时间\t, 交易金额\t, 交易余额\t, 收付标志\t, 交易对手账卡号\t, 现金标志\t, 对手户名\t, 对手身份证号\t, 对手开户银行\t, 摘要说明\t, 交易币种\t, 交易网点名称\t, 交易发生地\t, 交易是否成功\t, 传票号\t, IP地址\t, MAC地址\t, 对手交易余额\t, 交易流水号\t, 日志号\t, 凭证种类\t, 凭证号\t, 交易柜员号\t, 备注\t, 交易类型\t)    11
(开户名称\t, 证照类型\t, 证照号码\t, 联系电话\t)                                                                                                                                                                                                                                       1
dtype: int64


In [6]:

group_流水2 = grouped.get_group(groupname_list[0])
group_流水2['文件序号I'] = group_流水2.index

from openpyxl import load_workbook
book = load_workbook("E:\\7.3徐州网赌\\文件目录.xlsx")
writer = pd.ExcelWriter("E:\\7.3徐州网赌\\文件目录.xlsx", engine='openpyxl')
writer.book = book
group_流水2.to_excel(writer, sheet_name ='流水文件目录-交易场所渠道少两列2',index = None)
writer.save()

print('Done!')

D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Done!


In [15]:
import numpy as np
import re
import datetime

df_忽略行 = pd.DataFrame()
df_冲正 = pd.DataFrame()
all_data = pd.DataFrame()

for i in range(len(file_list)):
    if file_list[i] in list(group_流水2['Path']):  #查控平台流水缺两列
        if type1 in file_list[i]:
            df = pd.read_excel(file_list[i],encoding = "gbk")
        elif type2 in file_list[i]:
            df = pd.read_csv(file_list[i], engine="python")
        df.columns = df.columns.str.strip()
        col_name = df.columns.tolist()
        
        df['交易场所'] = df['凭证号']
        df['渠道'] = df['凭证号']
        
        df['path'] = file_list[i]
        df['序号'] = df.index
        df = df.applymap(lambda x: re.sub('\t','',str(x)))
        df['交易卡号'] = df['交易卡号'].map(lambda x: np.NaN if str(x)=='' else x)
        df['交易卡号'][df['交易卡号'].isnull()]=df['交易账号'][df['交易卡号'].isnull()] # 账号填充卡号空白
        df_忽略行 = df_忽略行.append(df[(df['交易金额']=='') | (df['交易卡号']=='') ])
        df = df.drop(df[(df['交易金额']=='') | (df['交易卡号']=='')].index)
        df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
        df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
        df['交易余额'] = df['交易余额'].map(lambda x: float(x))  #交易余额指定浮点型
        df = df.drop_duplicates(subset = col_name[0:16],keep = 'last')  #去除重复
        df = df.sort_values(by = ['交易卡号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:]
        print(str(i)+'. 单表去重后记录数:'+str(df.shape[0]))
        df_负 = df[df['交易金额']<0]
        for j in range(0,len(df_负)):
            df_正 = df[(df['交易金额']== abs(df_负['交易金额'].iloc[j])) & \
                (df['收付标志']== df_负['收付标志'].iloc[j]) & \
                (df['交易时间']<= df_负['交易时间'].iloc[j]) & \
                (df['交易卡号']== df_负['交易卡号'].iloc[j]) & \
                (
                 (df['交易对手账卡号']== df_负['交易对手账卡号'].iloc[j]) | \
                 ((df['交易对手账卡号'] == '') & (df_负['交易对手账卡号'].iloc[j] == '')) | \
                 ((df['交易对手账卡号'] != '') & (df_负['交易对手账卡号'].iloc[j] == '')) | \
                 ((df['交易对手账卡号'] == '') & (df_负['交易对手账卡号'].iloc[j] != '')) | \
                 ((df['交易对手账卡号'] != '') & (df['交易卡号']==df_负['交易对手账卡号'].iloc[j]))
                )
                ]
            d_labels = [df_正.index.max(),df_负.index[j]]  #冲正匹配记录的索引
            if len(df_正)>0:
                df_冲正 = df_冲正.append(df.loc[d_labels])
                df = df.drop(d_labels)
                print(d_labels)
            elif len(df_正)==0:
                print(d_labels)
        print('剔除冲正后保留记录数:'+str(df.shape[0]))
        all_data = all_data.append(df)

all_data.to_csv("E:\\7.3徐州网赌\\流水合并剔除冲正.csv",index = None)
print(all_data.shape)

book = load_workbook("E:\\7.3徐州网赌\\剔除行.xlsx")
writer = pd.ExcelWriter("E:\\7.3徐州网赌\\剔除行.xlsx", engine='openpyxl')
writer.book = book

df_冲正.to_excel(writer, sheet_name='冲正全5',index = None)
df_忽略行.to_excel(writer, sheet_name='忽略行5',index = None)

writer.save()


0. 单表去重后记录数:1972
剔除冲正后保留记录数:1972
1. 单表去重后记录数:4739
[1759, 1760]
[2706, 2707]
[2708, 2709]
[2710, 2711]
[2744, 2745]
[2746, 2747]
[3363, 3364]
剔除冲正后保留记录数:4725
2. 单表去重后记录数:933
[44, 45]
剔除冲正后保留记录数:931
3. 单表去重后记录数:15320
[1300, 1301]
[2157, 2158]
[2159, 2160]
[2628, 2630]
[2627, 2635]
[2626, 2636]
[2629, 2637]
[2731, 2732]
[2783, 2784]
[2785, 2787]
[nan, 2893]
[4212, 4213]
[4305, 4306]
[4307, 4308]
[4367, 4368]
[nan, 4451]
[nan, 4454]
[4545, 4546]
[4586, 4593]
[4584, 4594]
[4590, 4596]
[4592, 4597]
[4647, 4650]
[nan, 4707]
[4748, 4749]
[4769, 4770]
[4771, 4772]
[4787, 4788]
[4874, 4875]
[5092, 5093]
[5094, 5095]
[5096, 5098]
[5097, 5099]
[5323, 5324]
[5558, 5559]
[5714, 5731]
[5930, 5948]
[5996, 6012]
[6154, 6155]
[6252, 6267]
[6333, 6334]
[6370, 6371]
[6404, 6405]
[6427, 6432]
[6529, 6538]
[6557, 6558]
[6610, 6611]
[6643, 6644]
[6798, 6824]
[6839, 6840]
[7240, 7241]
[7264, 7266]
[7283, 7284]
[7292, 7297]
[7457, 7459]
[7531, 7532]
[7539, 7543]
[7676, 7677]
[7737, 7752]
[7870, 7871]
[7873, 78

D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


6. 单表去重后记录数:3956
剔除冲正后保留记录数:3956
8. 单表去重后记录数:697
剔除冲正后保留记录数:697
9. 单表去重后记录数:265
剔除冲正后保留记录数:265
10. 单表去重后记录数:4367
[1883, 1884]
[1905, 1906]
[2701, 2702]
剔除冲正后保留记录数:4361
11. 单表去重后记录数:3449
剔除冲正后保留记录数:3449
(49382, 34)


In [19]:
df = pd.read_csv('E:\\7.3徐州网赌\\第四阶段分析结果\\第四批清洗\\流水户名.csv',encoding = 'utf-8',dtype=object)
df = df.applymap(lambda x: re.sub('nan','',str(x)))
df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
df['交易余额'] = df['交易余额'].map(lambda x: float(x))
dfs = all_data.append(df)
dfs = dfs.sort_values(by = ['交易卡号','交易时间','交易金额'],ascending=[True,True,False]).reset_index().iloc[:,1:]

account2 = pd.read_excel('E:\\7.3徐州网赌\\银行卡人员信息整理(提供给公司).xls',sheet_name = '去重',encoding = 'gbk')
dfs2 = pd.merge(dfs,account2.drop_duplicates(subset='银行卡号'),how = 'left',left_on='交易卡号',right_on='银行卡号')
dfs2['交易户名'][dfs2['姓名'].notnull()] = dfs2['姓名'][dfs2['姓名'].notnull()]

df = dfs2.iloc[:,:34]
df.columns = all_data.columns
print(df.shape)

D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


(1986859, 34)


In [18]:
# df[(df['交易卡号']=='') | (df['交易卡号']=='nan')]

,交易卡号,交易账号,查询反馈结果原因,交易户名,交易证件号码,交易时间,交易金额,交易余额,收付标志,交易对手账卡号,...,日志号,凭证种类,凭证号,交易柜员号,备注,交易类型,交易场所,渠道,path,序号
0,,,查询无明细,,,,0.0,0.0,,,...,,,,,,,,,E:\7.3徐州网赌\20200626经侦来的数据原件总\中国民生银行股份有限公司交易明细信...,0


In [20]:

df_余额 = pd.DataFrame(columns = ['交易卡号','交易户名','交易时间','交易余额'])
df_S1 = df.groupby('交易卡号').size().reset_index()
df_S2 = pd.DataFrame(columns = ['交易卡号','Size','False','True','最早时间','最晚时间'])
for i in df_S1.index:
    df_every = df.groupby('交易卡号').get_group(df_S1['交易卡号'][i])
    bool_S = df_every.duplicated(subset = col_name[0:16],keep = 'last').value_counts()
    if len(bool_S)==1:
        df_S2.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],'Size':df_every.shape[0],'False':bool_S[0],'True':0,'最早时间':df_every['交易时间'].min(),'最晚时间':df_every['交易时间'].max()}
    elif len(bool_S)==2:
        df_S2.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],'Size':df_every.shape[0],'False':bool_S[0],'True':bool_S[1],'最早时间':df_every['交易时间'].min(),'最晚时间':df_every['交易时间'].max()}
    df_余额.loc[i] = df_every[['交易卡号','交易户名','交易时间','交易余额']].loc[df_every.index.max()]

with pd.ExcelWriter("E:\\7.3徐州网赌\\单卡统计.xlsx") as writer:
    df_S2.to_excel(writer,sheet_name = '单卡去重统计',index = None)
    df_余额.to_excel(writer,sheet_name = '单卡最后一笔交易余额',index = None)
# print(df_S1)
print(df_S2)
print(df.duplicated(subset = col_name[0:16]).value_counts())
df = df.drop_duplicates(subset = col_name[0:16],keep = 'last')
df.to_csv("E:\\7.3徐州网赌\\流水户名.csv",index = None)

                          交易卡号  Size False True                 最早时间  \
0                 100148127645   153   153    0  2020-01-02 11:17:13   
1     1001810033940469_C100CNY     7     7    0  2020-01-25 01:38:07   
2     1001810044484698_C100CNY   211   211    0  2020-01-02 21:04:32   
3     1001810069314478_C100CNY  1000  1000    0  2020-03-03 14:57:39   
4     1001810073566360_C100CNY  1001  1001    0  2020-03-02 16:39:34   
...                        ...   ...   ...  ...                  ...   
1950      95040078801200000557     1     1    0  2020-01-21 23:50:11   
1951      95040078801900000558     1     1    0  2020-02-28 23:40:53   
1952       9558801001151426019   512   512    0  2020-01-01 01:41:21   
1953       9559980681566109119   152   152    0  2020-02-10 23:41:12   
1954       9559982021721381410  1318  1318    0  2020-01-01 21:48:12   

                     最晚时间  
0     2020-07-15 09:33:59  
1     2020-06-21 00:59:38  
2     2020-06-05 00:24:33  
3     2020-05-25 01:52:

In [1]:
df_卡号 = df[(df['交易卡号']!=df['交易账号']) & (df['交易账号']!='')]
df_a1 = df_卡号.groupby(['交易卡号','交易账号']).size().reset_index()
df_a2 = df_a1.groupby(['交易卡号'],as_index = False).size()
card = df_a1[df_a1['交易卡号'].isin(df_a2[df_a2==1]).index]
card.to_excel("E:\\7.3徐州网赌\\卡号账号.xlsx",index = None)
print('Done!')
dfs0 = pd.merge(df,card,how = 'left',on='交易账号')
print(dfs0)
dfs0['交易卡号_x'][(dfs0['交易卡号_x']!=dfs0['交易卡号_y']) & dfs0['交易卡号_y'].notnull()]=dfs0['交易卡号_y'][dfs0['交易卡号_x']!=dfs0['交易卡号_y'] & dfs0['交易卡号_y'].notnull()]
df = dfs0.iloc[:,:34]
df.columns = all_data.columns
df.to_csv("E:\\7.3徐州网赌\\流水户名.csv",index = None)
print(df.shape)

NameError: name 'df' is not defined

In [25]:
# df_户名 = df[(df['交易对手账卡号'].isin(df_余额['交易卡号'][df_余额['交易户名']==''])) & (df['对手户名']!='')]
# df_a1 = df_户名.groupby(['交易对手账卡号','对手户名']).size().reset_index()
# df_a2 = df_a1.groupby(['交易对手账卡号'],as_index=False).size()
# account3 = df_a1[df_a1['交易对手账卡号'].isin(df_a2[df_a2==1].index)]
# account3.to_excel("E:\\7.3徐州网赌\\户名.xlsx",index = None)
# print('Done!')

Done!


In [26]:
# dfs3 = pd.merge(df,account3,how = 'left',left_on='交易卡号',right_on='交易对手账卡号')
# dfs3

,交易卡号,交易账号,查询反馈结果原因,交易户名,交易证件号码,交易时间,交易金额,交易余额,收付标志,交易对手账卡号_x,...,交易柜员号,备注,交易类型,交易场所,渠道,path,序号,交易对手账卡号_y,对手户名_y,0
0,100148127645,100148127645,成功,,,2020-01-02 11:17:13,18302.5,83091.91,出,6217855000069199701,...,9880100,,转账支出,,,E:\7.3徐州网赌\Q9案 7.20数据\中国银行交易明细信息.csv,6590,100148127645,安国市百味堂中药材有限公司,3.0
1,100148127645,100148127645,成功,,,2020-01-02 14:44:01,14566.0,68525.91,出,6013825002008065860,...,9880100,,转账支出,,,E:\7.3徐州网赌\Q9案 7.20数据\中国银行交易明细信息.csv,6591,100148127645,安国市百味堂中药材有限公司,3.0
2,100148127645,100148127645,成功,,,2020-01-04 09:14:34,27440.0,41085.91,出,6217855000005531660,...,9880100,,转账支出,,,E:\7.3徐州网赌\Q9案 7.20数据\中国银行交易明细信息.csv,6592,100148127645,安国市百味堂中药材有限公司,3.0
3,100148127645,100148127645,成功,,,2020-01-05 09:55:35,23502.0,17583.91,出,6217855000020669057,...,9880100,,转账支出,,,E:\7.3徐州网赌\Q9案 7.20数据\中国银行交易明细信息.csv,6593,100148127645,安国市百味堂中药材有限公司,3.0
4,100148127645,100148127645,成功,,,2020-01-05 10:10:45,100000.0,117583.91,进,6222080409004629168,...,9880809,ibps.101 C20002001R,网上支付,,,E:\7.3徐州网赌\Q9案 7.20数据\中国银行交易明细信息.csv,6594,100148127645,安国市百味堂中药材有限公司,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1986854,9559982021721381410,911801100159325,,李美霞,,2020-06-14 19:32:54,1388.6,1456.03,进,991290001018,...,,银联入账,INET,,,E:\7.3徐州网赌\20200626经侦来的数据原件总\（552号）中国农业银行交易明细信...,70810,NaN,NaN,NaN
1986855,9559982021721381410,911801100159325,,李美霞,,2020-06-14 19:34:03,1400.0,56.03,出,6222350082689140,...,,转支,IBPS,,,E:\7.3徐州网赌\20200626经侦来的数据原件总\（552号）中国农业银行交易明细信...,70811,NaN,NaN,NaN
1986856,9559982021721381410,911801100159325,,李美霞,,2020-06-14 19:38:11,5785.0,5841.03,进,991290001018,...,,银联入账,INET,,,E:\7.3徐州网赌\20200626经侦来的数据原件总\（552号）中国农业银行交易明细信...,70812,NaN,NaN,NaN
1986857,9559982021721381410,911801100159325,,李美霞,,2020-06-14 19:48:50,3741.8,2099.23,出,215500690,...,,支付宝,EPAY,,,E:\7.3徐州网赌\20200626经侦来的数据原件总\（552号）中国农业银行交易明细信...,70813,NaN,NaN,NaN


In [27]:
# dfs3['交易户名'][dfs3['对手户名_y'].notnull()] = dfs3['对手户名_y'][dfs3['对手户名_y'].notnull()]

# df = dfs3.iloc[:,:34]
# df.columns = all_data.columns
# df.to_csv("E:\\7.3徐州网赌\\流水户名.csv",index = None)
# print(df.shape)


D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(1986859, 34)


In [29]:

box1 = df[df['交易对手账卡号'].isin(df[df['path'].isin(file_list)]['交易卡号']) |(df['path'].isin(file_list))]
col_2 = box1.columns.tolist()+['付款方','收款方']
box2 = pd.DataFrame(box1,columns = col_2)
print(box2.shape)
box2['付款方'][box2['收付标志']=='出'] = box2['交易卡号'][box2['收付标志']=='出']
box2['收款方'][box2['收付标志']=='出'] = box2['交易对手账卡号'][box2['收付标志']=='出']
box2['付款方'][box2['收付标志']=='进'] = box2['交易对手账卡号'][box2['收付标志']=='进']
box2['收款方'][box2['收付标志']=='进'] = box2['交易卡号'][box2['收付标志']=='进']

box_group = box2.groupby(['付款方','收款方'])
box_group.size().to_excel("E:\\7.3徐州网赌\\box.xlsx")
print(len(box_group.size().reset_index()))

(49551, 36)


D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


4345


In [30]:
# 前提是调取的数据以日为最小单位，实际调取至今的话“至今”最小单位时分秒，有误差，例如6217003800028006371
# 查控平台数据收付标志有误，例如6230720210512159709 曾路恒

box_AB = pd.DataFrame()
df_AB = df
for i in range(0,len(box_group.size().reset_index())):
        box_every = box_group.get_group(box_group.size().index[i])
        box_every['日期'] = box_every['交易时间'].str[0:10]

        if len(box_every.drop_duplicates(['交易卡号']))==2:
            if box_every[box_every['收付标志']=='出']['日期'].min()>box_every[box_every['收付标志']=='进']['日期'].min():
                box_every = box_every[box_every['日期']>=box_every[box_every['收付标志']=='出']['日期'].min()]
            else:
                box_every = box_every[box_every['日期']>=box_every[box_every['收付标志']=='进']['日期'].min()]
            if len(box_every.drop_duplicates(['交易卡号']))==2:
                if box_every[box_every['收付标志']=='出']['日期'].max()<box_every[box_every['收付标志']=='进']['日期'].max():
                    box_every = box_every[box_every['日期']<=box_every[box_every['收付标志']=='出']['日期'].max()]
                else:
                    box_every = box_every[box_every['日期']<=box_every[box_every['收付标志']=='进']['日期'].max()]
                box_AB = box_AB.append(box_every)
                df_AB = df_AB.drop(box_every[box_every['收付标志']=='出'].index)
                print(str(i)+'去重后保留'+str(df_AB.shape[0]))
            else:
                print(str(i)+'无交集')
        else:
                print(i)
df_AB.to_csv("E:\\7.3徐州网赌\\流水去重后.csv",index = None)

book = load_workbook("E:\\7.3徐州网赌\\流水去重匹配.xlsx")
writer = pd.ExcelWriter("E:\\7.3徐州网赌\\流水去重匹配.xlsx", engine='openpyxl')
writer.book = book
box_AB.sort_values(by = ['付款方','收款方','交易时间','收付标志'],ascending=True).to_excel(writer, sheet_name ='5')

writer.save()

# box_AB.sort_values(by = ['付款方','收款方','交易时间','收付标志'],ascending=True).to_excel("E:\\7.3徐州网赌\\流水去重匹配.xlsx") 

print(df_AB.shape)
print(box_AB.shape)
print('Done!')

D:\Users\ThinkPad\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065


3493
3494
3495
3496
3497
3498
3499
3500
3501
3502
3503
3504
3505
3506
3507
3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667去重后保留1986217
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
369

In [32]:
df = df_AB[df_AB['对手户名']!='']
df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
df['交易余额'] = df['交易余额'].map(lambda x: float(x))
df_set2 = df[(df['对手户名'].str.contains('公司')) | (df['交易户名'].str.contains('公司'))]
df_set2.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\公司.xlsx")
print('Done!')

Done!


In [33]:
df = df_AB
set3= ['6228482698441427571']
df_set3 = df[(df['交易卡号'].isin(set3)) | (df['交易对手账卡号'].isin(set3))]
df_set3.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\张俊生.xlsx")

In [34]:

df = df_AB
set3= ['6225180100003835']
df_set3 = df[(df['交易卡号'].isin(set3)) | (df['交易对手账卡号'].isin(set3))]
df_set3.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\冯铭轩.xlsx")

In [35]:
df = df_AB
set3= ['41018200040022986']
df_set3 = df[(df['交易卡号'].isin(set3)) | (df['交易对手账卡号'].isin(set3))]
df_set3.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\深圳市海荣杰信息咨询有限公司.xlsx")


In [6]:
import pandas as pd
df = pd.read_csv("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批清洗\\流水去重后.csv",encoding = 'utf-8',dtype = object)
df_cut = df[df['交易对手账卡号'].isin(df['交易卡号'])]
df_cut.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\主账号.xlsx")
print('Done!')

Done!


In [7]:
df_cut[(df_cut['交易时间'].str[0:7]=='2020-02') | (df_cut['交易时间'].str[0:7]=='2020-03')].to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\主账号2月3月.xlsx")

In [8]:
df[(df['交易时间'].str[0:7]=='2020-02') | (df['交易时间'].str[0:7]=='2020-03')].to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\2月3月.xlsx")

In [45]:
# import pandas as pd
# import numpy as np
# import re

# df = pd.read_csv("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批清洗\\流水户名.csv",encoding = 'utf-8',dtype = object)
# df = df.applymap(lambda x: re.sub('nan','',str(x)))
# df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
# df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
# df['交易余额'] = df['交易余额'].map(lambda x: float(x))
# df_S1 = df.groupby('交易卡号').size().reset_index()

# df_S9 = pd.DataFrame(columns = ['交易卡号','交易户名','Size','False','True','最早时间','最晚时间','总和','净和','入向','出向','夜间交易额','入向对手数','出向对手数','收付检验起始值','Path'])
df_S9 = pd.DataFrame(columns = ['交易卡号','交易户名','Size','False','True','最早时间','最晚时间','余额','总和','净和','入向','出向','入向对手数','出向对手数','收付检验起始值','Path'])
for i in df_S1.index:
    df_every = df.groupby('交易卡号').get_group(df_S1['交易卡号'][i])
    bool_S = df_every.duplicated(subset = df.columns[0:16],keep = 'last').value_counts()
    df_night = df_every[df_every['交易时间'].str[11:13].isin(['00','01','02','03','04','05','06'])]
    df_S9.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],
                            '交易户名':df_every['交易户名'].iloc[0],
                            'Size':df_every.shape[0],
                            'False':bool_S[0],
                            'True':bool_S.map(lambda x:0 if len(bool_S)==1 else bool_S[1]).values[0],
                            '最早时间':df_every['交易时间'].min(),
                            '最晚时间':df_every['交易时间'].max(),
                            '余额':df_every['交易余额'].loc[df_every.index.max()],
                            '总和':df_every['交易金额'].sum(),
                            '净和':df_every['交易金额'][df_every['收付标志']=='进'].sum()-df_every['交易金额'][df_every['收付标志']=='出'].sum(),
                            '入向':df_every['交易金额'][df_every['收付标志']=='进'].sum(),
                            '出向':df_every['交易金额'][df_every['收付标志']=='出'].sum(),
#                             '夜间交易额':df_night.apply(lambda x:0 if len(df_night)==0 else x['交易金额']).sum()
                            '入向对手数':len(df_every['交易对手账卡号'][df_every['收付标志']=='进'].drop_duplicates()),
                            '出向对手数':len(df_every['交易对手账卡号'][df_every['收付标志']=='出'].drop_duplicates()),
                            '收付检验起始值':df_every['交易余额'].iloc[0]+df_every.apply(lambda x: x['交易金额'] if x['收付标志']=='出' else -x['交易金额'],axis=1).iloc[0],
                            'Path':df_every['path'].iloc[0]
               }

    print(i)
df_S9.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\交易统计.xlsx",index = None)
print('Done!')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
Done!


In [38]:
df_every = df.groupby('交易卡号').get_group('10069723060001890750500100061426')

df_every.loc[:,['交易时间','交易金额','交易余额','收付标志']]

交易时间    2020-06-30 14:26:59
交易余额                 389010
收付标志                      出
Name: 6185, dtype: object

In [43]:
df_every.apply(lambda x: x['交易金额'] if x['收付标志']=='出' else -x['交易金额'],axis=1)

5857   -16656494.84
5858    10000000.00
5859    20475000.00
5860    20000000.00
5861     8000000.00
           ...     
6181   -12009336.98
6182    20000000.00
6183    -1000778.08
6184     1000000.00
6185     8000000.00
Length: 329, dtype: float64

In [42]:
df_every['交易余额'].iloc[0]+df_every.apply(lambda x: x['交易金额'] if x['收付标志']=='出' else -x['交易金额'],axis=1).iloc[0]

218884.4299999997

In [37]:
bool_S.map(lambda x:0 if len(bool_S)==1 else bool_S[1]).values[0]

0

In [57]:
df_night['交易金额'].append(pd.Series({'交易金额':0}))

交易金额    0
dtype: int64

In [68]:
df_night.applymap(lambda x:0 if len(df_night)==0 else df_night['交易金额']).sum()

交易卡号        1985761     23919.08
1985904     49050.72
1986...
交易账号        1985761     23919.08
1985904     49050.72
1986...
查询反馈结果原因    1985761     23919.08
1985904     49050.72
1986...
交易户名        1985761     23919.08
1985904     49050.72
1986...
交易证件号码      1985761     23919.08
1985904     49050.72
1986...
交易时间        1985761     23919.08
1985904     49050.72
1986...
交易金额        1985761     23919.08
1985904     49050.72
1986...
交易余额        1985761     23919.08
1985904     49050.72
1986...
收付标志        1985761     23919.08
1985904     49050.72
1986...
交易对手账卡号     1985761     23919.08
1985904     49050.72
1986...
现金标志        1985761     23919.08
1985904     49050.72
1986...
对手户名        1985761     23919.08
1985904     49050.72
1986...
对手身份证号      1985761     23919.08
1985904     49050.72
1986...
对手开户银行      1985761     23919.08
1985904     49050.72
1986...
摘要说明        1985761     23919.08
1985904     49050.72
1986...
交易币种        1985761     23919.08
1985904     49050.72
1986...
交易网点名称  

In [64]:
df_night

,交易卡号,交易账号,查询反馈结果原因,交易户名,交易证件号码,交易时间,交易金额,交易余额,收付标志,交易对手账卡号,...,日志号,凭证种类,凭证号,交易柜员号,备注,交易类型,交易场所,渠道,path,序号
1985761,9559982021721381410,911801100159325,,李美霞,,2020-02-29 00:42:45,1039.96,2308.54,进,243300133,...,110984109,,,,代付,EPAY,,,E:\7.3徐州网赌\20200626经侦来的数据原件总\（552号）中国农业银行交易明细信...,69717
1985904,9559982021721381410,911801100159325,,李美霞,,2020-03-31 00:08:59,2132.64,26067.37,进,991584000049,...,100918633,,,,银联入账,INET,,,E:\7.3徐州网赌\20200626经侦来的数据原件总\（552号）中国农业银行交易明细信...,69860
1986032,9559982021721381410,911801100159325,,李美霞,,2020-04-09 00:39:19,1000.00,919.82,出,6259063547185663,...,108654052,,,,转支,IBPS,,,E:\7.3徐州网赌\20200626经侦来的数据原件总\（552号）中国农业银行交易明细信...,69988
1986033,9559982021721381410,911801100159325,,李美霞,,2020-04-09 00:41:02,900.00,19.82,出,6222350082655570,...,109548590,,,,转支,IBPS,,,E:\7.3徐州网赌\20200626经侦来的数据原件总\（552号）中国农业银行交易明细信...,69989
1986094,9559982021721381410,911801100159325,,李美霞,,2020-04-18 01:10:51,5000.00,1296.61,出,4367450027128111,...,113391388,,,,转支,IBPS,,,E:\7.3徐州网赌\20200626经侦来的数据原件总\（552号）中国农业银行交易明细信...,70050
1986095,9559982021721381410,911801100159325,,李美霞,,2020-04-18 01:11:42,200.00,1096.61,出,6228025706687100,...,113441819,,,,转支,IBPS,,,E:\7.3徐州网赌\20200626经侦来的数据原件总\（552号）中国农业银行交易明细信...,70051
1986096,9559982021721381410,911801100159325,,李美霞,,2020-04-18 01:12:48,300.00,796.61,出,6222533548586857,...,113508836,,,,转支,IBPS,,,E:\7.3徐州网赌\20200626经侦来的数据原件总\（552号）中国农业银行交易明细信...,70052
1986097,9559982021721381410,911801100159325,,李美霞,,2020-04-18 01:13:38,700.00,96.61,出,6228100043420430,...,113556681,,,,转支,IBPS,,,E:\7.3徐州网赌\20200626经侦来的数据原件总\（552号）中国农业银行交易明细信...,70053
1986116,9559982021721381410,911801100159325,,李美霞,,2020-04-19 00:45:48,1000.00,2283.63,出,99999999999999999,...,110978724,,,,财付通,EPAY,,,E:\7.3徐州网赌\20200626经侦来的数据原件总\（552号）中国农业银行交易明细信...,70072
1986129,9559982021721381410,911801100159325,,李美霞,,2020-04-20 00:00:00,435.30,593.49,进,991290001018,...,103236946,,,,银联入账,INET,,,E:\7.3徐州网赌\20200626经侦来的数据原件总\（552号）中国农业银行交易明细信...,70088


In [35]:
bool_A = pd.Series({'False':155,'True':10})
print(bool_A.map(lambda x:0 if len(bool_A)==1 else bool_A[1]).values[0])
bool_A

10


False    155
True      10
dtype: int64

In [36]:
bool_B = pd.Series({'False':155})
print(bool_B.map(lambda x:0 if len(bool_B)==1 else bool_B[1]).values[0])
bool_B

0


False    155
dtype: int64

In [21]:
len(df_every[df_every['交易时间'].str[11:13].isin(['00','01','02','03','04','05','06'])])==0

True

In [24]:
if len(df_every[df_every['交易时间'].str[11:13].isin(['00','01','02','03','04','05','06'])])==0:
    print(0)
else:
    print(df_every['交易金额'][df_every['交易时间'].str[11:13].isin(['00','01','02','03','04','05','06'])].sum())        


0


In [65]:
bool_S = df_every.duplicated(subset = df.columns[0:16],keep = 'last').value_counts()
bool_S

False    153
dtype: int64

In [52]:
pd.Series(bool_S)

False    153
dtype: int64

In [59]:
bool_S.map(lambda x:pd.Series(bool_S)).values

array([False    153
dtype: int64], dtype=object)

In [58]:
len(pd.Series(bool_S))==1

True

In [76]:
bool_S.map(lambda x:0 if len(pd.Series(bool_S))==1 else x).values[0]

0

In [68]:
type(bool_S.map(lambda x:0 if len(pd.Series(bool_S))==1 else x))

pandas.core.series.Series

In [2]:
import pandas as pd
import numpy as np
import re
df = pd.read_csv("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批清洗\\流水去重后.csv",encoding = 'utf-8',dtype = object)
df = df.applymap(lambda x: re.sub('nan','',str(x)))
df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
df['交易余额'] = df['交易余额'].map(lambda x: float(x))
import math
df_lst = pd.DataFrame()
lst = [4850,5000]
for i in range(0,len(lst)):
        df_lst1 = df[df['交易金额'].map(lambda x: (math.floor(x) % lst[i] ==0) & (x>1) & (x>math.floor(x)) & (x<math.ceil(x)))]
        df_lst = df_lst.append(df_lst1)
df_lst.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\特征值.xlsx")
print('Done!')

Done!


In [1]:
import pandas as pd
import numpy as np
import re
df = pd.read_csv("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批清洗\\流水去重后.csv",encoding = 'utf-8',dtype = object)
df = df.applymap(lambda x: re.sub('nan','',str(x)))
df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
df['交易余额'] = df['交易余额'].map(lambda x: float(x))
import math
df_lst2 = pd.DataFrame()
lst2 = [4840]
df_lst2 = df[df['交易金额'].map(lambda x: (math.floor(x) % lst2[0] ==0) & (x>1))]

df_lst2.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\特征值返还.xlsx")
print('Done!')

Done!


In [7]:
set1 = ['6230520460180575270','6212254402003530645','6230520460180529574','6217003110031459181','6230520460174815179','6230520460201120072','6228480809474199779','6228480688620600579','6221806510001348804','6214624721000509115','6217730304705120','6230520460172424875','6217680905346867','6217261028013139505','6217996510031881814','6217995540024422087']

df_set1 = df[(df['交易卡号'].isin(set1)) | (df['交易对手账卡号'].isin(set1))]
df_set1.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\卖分收款卡.xlsx")
print('Done!')

Done!


In [9]:
set1_1 = ['622439380006428724','6217003800030672970','6226192012537858','6270430501000272058','6217003800028933384','6221806510001349380','6222034000033456371','6217261028013139570','6270430501000214878','6214820130008872511','6217261028013139703','6217995930007625865','6226192011979861','6221806510001349042','6222031408000166456','6212264402092189210','6230720210512320277','6221805840000786006','6214832808241173','6221532320027770705','6214622121006404167','6230880006034459569','6212254402003530710','6228480469870757678','6226192013125505','6230520460201121377','6214832801974168','621726102801313957','6228482469595600979','6217003800028118333','6214624721000524999','6216693100007834464','6226192013214655','6217007200089350592','6212264402095546705','6214820130009540042','6217003800030672962','6214624721000509115','6217003800027951270','6217730304705120','6217995540024054112','6217003800028004046','6217680905346867','6217261028013139505','6230520460172424875','6230520460174815179','6230520460180529574','6221806510001348804','6217003110031459181','6228480809474199779','6217996510031881814','6230520460201120072','6212254402003530645','6228480688620600579','6230520460180575270','6217995540024422087']
df_set1_1 = df[(df['交易卡号'].isin(set1_1)) | (df['交易对手账卡号'].isin(set1_1))]
df_set1_1.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\卖分收款卡_1.xlsx")
print('Done!')

Done!


In [12]:
set3 = ['6217997010016305185','6230522060020868279','6230520460196201077','263769472463','1408868209000034475','6217003800028006371','6228410604505354377','6216693100007820612','6228480158286454372','081101040051326','6217852600023663891','6217261028013139521','6217730809339912','41050165284000000284','6212264402098270246','12081101040051326','6226631703883873','6229466849508937','6236683520006800689','6217003160008355433','6228480158286570177','6227001832590145353','6230720210512317984','6214680057138677_001','6230521980027421775','6223226033841965','6236683520007474484','6217856100094373948','6217003520012027452','6230520050052814673','6230880010013223624','6223093310013803963','161037691','6230582000028437898','6236602300363450','101015765448821','6217996400040795405','6228480626733518775','6236683320027070549','6236682950007406320','6217261028013123327','6217000010122681615','6232082800008700608','6230580000076818199','6212263602137714663']
df_set3 = df[(df['交易卡号'].isin(set3)) | (df['交易对手账卡号'].isin(set3))]
df_set3.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\资金去向.xlsx")
print('Done!') 

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [NoneType]

In [14]:
set2 = ['622439380006428724','6217003800030672970','6226192012537858','6270430501000272058','6217003800028933384','6221806510001349380','6222034000033456371','6217261028013139570','6270430501000214878','6214820130008872511','6217261028013139703','6217995930007625865','6226192011979861','6221806510001349042','6222031408000166456','6212264402092189210','6230720210512320277','6221805840000786006','6214832808241173','6221532320027770705','6214622121006404167','6230880006034459569','6212254402003530710','6228480469870757678','6226192013125505','6230520460201121377','6214832801974168','621726102801313957','6228482469595600979','6217003800028118333','6214624721000524999','6216693100007834464','6226192013214655','6217007200089350592','6212264402095546705','6214820130009540042','6217003800030672962','6214624721000509115','6217003800027951270','6217730304705120','6217995540024054112','6217003800028004046','6217680905346867','6217261028013139505','6230520460172424875','6230520460174815179','6230520460180529574','6221806510001348804','6217003110031459181','6228480809474199779','6217996510031881814','6230520460201120072','6212254402003530645','6228480688620600579','6230520460180575270','6217995540024422087','6217997010016305185','6230522060020868279','6230520460196201077','263769472463','1408868209000034475','6217003800028006371','6228410604505354377','6216693100007820612','6228480158286454372','081101040051326','6217852600023663891','6217261028013139521','6217730809339912','41050165284000000284','6212264402098270246','12081101040051326','6226631703883873','6229466849508937','6236683520006800689','6217003160008355433','6228480158286570177','6227001832590145353','6230720210512317984','6214680057138677_001','6230521980027421775','6223226033841965','6236683520007474484','6217856100094373948','6217003520012027452','6230520050052814673','6230880010013223624','6223093310013803963','161037691','6230582000028437898','6236602300363450','101015765448821','6217996400040795405','6228480626733518775','6236683320027070549','6236682950007406320','6217261028013123327','6217000010122681615','6232082800008700608','6230580000076818199','6212263602137714663']
df_set2 = df[(df['交易卡号'].isin(set2)) | (df['交易对手账卡号'].isin(set2))]
df_set2.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\资金去向(合并23).xlsx")
print('Done!') 

Done!


In [2]:
# 6214680057138677_001
setn = ['6230520460196201077','6228480626733518775','6217261028013139521','6212264402098270246','6217003520012027452','6217261028013139570','6236682950007406320','6236683520006800689']
df_setn = df[(df['交易卡号'].isin(setn)) | (df['交易对手账卡号'].isin(setn))]
df_setn.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\卡商碰撞).xlsx")
print('Done!') 


Done!


In [1]:
import pandas as pd
import numpy as np
import re
df = pd.read_csv("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批清洗\\流水户名.csv",encoding = 'utf-8',dtype = object)
df = df.applymap(lambda x: re.sub('nan','',str(x)))
df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
df['交易余额'] = df['交易余额'].map(lambda x: float(x))
df_纳盟 = pd.read_excel("E:\\7.3徐州网赌\\20200728公司\\河南纳盟商贸有限公司开户至今交易流水及对手信息.xlsx",encoding = 'gbk',dtype = object)
df_国宏 = pd.read_excel("E:\\7.3徐州网赌\\20200728公司\\深圳市国宏人力资源有限公司.xlsx",sheet_name = '代付出账明细',encoding = 'gbk',dtype = object)

list_1 = df_纳盟['对方帐号或卡号'][df_纳盟['借贷标志']=='C'].drop_duplicates().tolist()
list_2 = df_国宏['encrypt_card_no'].drop_duplicates().tolist()

set = list_1+list_2
df_setn = df[df['交易对手账卡号'].isin(set)]
df_setn.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\卡商相关).xlsx")
print('Done!') 


Done!


In [4]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批清洗\\流水户名.csv",encoding = 'utf-8',dtype = object)
df = df.applymap(lambda x: re.sub('nan','',str(x)))
df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
df['交易余额'] = df['交易余额'].map(lambda x: float(x))
df_S1 = df.groupby('交易卡号').size().reset_index()

# df_S9 = pd.DataFrame(columns = ['交易卡号','交易户名','Size','False','True','最早时间','最晚时间','总和','净和','入向','出向','夜间交易额'])
df_S9 = pd.DataFrame(columns = ['交易卡号','交易户名','Size','False','True','最早时间','最晚时间','余额','总和','净和','入向','出向','入向对手数','出向对手数'])
for i in df_S1.index:
    df_every = df.groupby('交易卡号').get_group(df_S1['交易卡号'][i])
    bool_S = df_every.duplicated(subset = df.columns[0:16],keep = 'last').value_counts()
    df_night = df_every[df_every['交易时间'].str[11:13].isin(['00','01','02','03','04','05','06'])]
    df_S9.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],
                            '交易户名':df_every['交易户名'].iloc[0],
                            'Size':df_every.shape[0],
                            'False':bool_S[0],
                            'True':bool_S.map(lambda x:0 if len(bool_S)==1 else bool_S[1]).values[0],
                            '最早时间':df_every['交易时间'].min(),
                            '最晚时间':df_every['交易时间'].max(),
                            '余额':df_every['交易余额'].loc[df_every.index.max()],
                            '总和':df_every['交易金额'].sum(),
                            '净和':df_every['交易金额'][df_every['收付标志']=='进'].sum()-df_every['交易金额'][df_every['收付标志']=='出'].sum(),
                            '入向':df_every['交易金额'][df_every['收付标志']=='进'].sum(),
                            '出向':df_every['交易金额'][df_every['收付标志']=='出'].sum(),
#                             '夜间交易额':df_night.applymap(lambda x:0 if len(df_night)==0 else df_night['交易金额'].sum())
                            '入向对手数':len(df_every['交易对手账卡号'][df_every['收付标志']=='进'].drop_duplicates()),
                            '出向对手数':len(df_every['交易对手账卡号'][df_every['收付标志']=='出'].drop_duplicates())
               }
    print(i)
df_S9.to_excel("E:\\7.3徐州网赌\\交易统计.xlsx",index = None)
print('Done!')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
Done!


In [1]:
import pandas as pd
import numpy as np
import re
df = pd.read_csv("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批清洗\\流水去重后.csv",encoding = 'utf-8',dtype = object)
df = df.applymap(lambda x: re.sub('nan','',str(x)))
df['交易金额'] = df['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
df['交易余额'] = df['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
df['交易余额'] = df['交易余额'].map(lambda x: float(x))
set4 = ['6216698000002276825','6222621010021317286','6217856100030171174','6217852600023663891','6222081402007473410','6217906300023397683','6229466849508937','6228480158286570177','6217751124000810674','6227001252120141543','6217933375245030','6212261705019432239','6217857600009846333','6217993900073803821','6217923375530754','6217997010016305185','6216693600003716227','6216693600002837628','622908133033325138','6214685060373864_001','6214852901463888','6216698000008501648','6228480068821985873','6217996710001358963','6226620412137093','6217920353514131','6216911707086638','6230195940060014684','6212261905005722839','6216698000003104182','6212263602137714663','6230580000056516623','6230910399107875652','6222620757566888888','622908193013312811','6230580000076818199','6226192012872610','6214623721003512968','6235756200001052610','6230521540029353275','6217921463195035','6230520680083033677','6217001930031445761','6230880010013223624','6217856100094373948','6236602300363450']

df_set4 = df[(df['交易卡号'].isin(set4)) | (df['交易对手账卡号'].isin(set4))]
df_set4.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\资金去向(挑选34).xlsx")
print('Done!')

Done!


In [2]:
set5 =['6217996400040726756','6228480158282216478','6217001820035113640','6215620002001329832','6212253602020142123','6216912309621285','6216912309515636','6217973900001719085','6216601700000518138','6217002950112528300','6217730304729260','6228480688268616275','6217922652773996','6221805810001534051','6210991066000006990','623063000009029144','6217001830043057440','6216698000010904327','6212262504004394978','6236682950007001584','6213360686016863865','6222031702002912788','6217261028012030283','6214855791903319','6213360599900348077','6222623120002984265','6214623121002433532','6221803900001203344']
set5 = set5 + set4

df_set5 = df[(df['交易卡号'].isin(set5)) | (df['交易对手账卡号'].isin(set5))]
df_set5.to_excel("E:\\7.3徐州网赌\\第五阶段分析结果\\第五批汇总切片\\资金去向(5).xlsx")
print('Done!')

Done!


In [ ]:


set5 =['6228480158284010077','6230910199079589673','6230520680034978673','6228411154513664671','6214835541708648','6217714902059056','6217261028013139638','6214835541297063','6214864311931818','6230580000284764599','6230580000117554290','6228410924523962179','6222034000034830632','6666663805616577','6259553700020012','6228480688248088371','6227001892600041307','6217231408002964658','6230584000002333251



In [ ]:
df_set5 = df[][df['对手户名']=='李宝燕']

In [3]:
df.shape

(1986217, 34)